# Get Zoopla Data
Use the Zoopla API to pull current house listings

In [1]:
import os
import math
import pandas as pd
pd.set_option('display.max_columns', 100)
from zoopla import Zoopla

/home/robertv/anaconda3/envs/datascience/lib/python3.8/site-packages/marshmallow/schema.py:200: ChangedInMarshmallow3Warning: The dateformat option is renamed to datetimeformat in marshmallow 3.
  warnings.warn(


In [2]:
# Call an instance of Zoopla with the secret API key
zoopla_instance = Zoopla(api_key=os.environ['ZOOPLA_API_KEY'])

### Download property data for Nuneaton or Hinckley, including recently sold properties

First get total number of properties - we can only access a maximum of 100 results per call, so will need to call the API multiple times

In [3]:
AREA = 'Hinckley'
SAVE_FOLDER = os.path.join('data', 'raw')
PAGE_SIZE = 100

In [4]:
properties = zoopla_instance.property_listings({
    'area': AREA,
    'order_by': 'age',
    'listing_status': 'sale',
    'include_sold': '1',
    'summarised': 'yes',
    'page_size': PAGE_SIZE
})

number_properties = properties['result_count']

print(f'Number of {AREA} properties = {number_properties}')

Number of Hinckley properties = 292


Now get all properties for the area and concatenate into a dataframe

In [5]:
number_pages = math.ceil(number_properties/PAGE_SIZE)

for i in range(number_pages):
    properties = zoopla_instance.property_listings({
        'area': AREA,
        'order_by': 'age',
        'listing_status': 'sale',
        'include_sold': '1',
        'summarised': 'yes',
        'page_size': PAGE_SIZE,
        'page_number': i+1
    })
    
    zoopla_df_tmp = pd.DataFrame.from_records(properties.listing)
    
    # loop over pages and concatenate into a pandas dataframe
    if i == 0:
        zoopla_df = zoopla_df_tmp
    else:
        zoopla_df = pd.concat([zoopla_df, zoopla_df_tmp], axis=0)

### Quick check of the data

In [6]:
zoopla_df.head()

,details_url,agent_phone,description,agent_address,latitude,longitude,outcode,country_code,num_bathrooms,listing_status,property_type,listing_id,num_recepts,post_town,displayable_address,floor_plan,image_url,street_name,agent_name,county,price_modifier,first_published_date,country,last_published_date,price,category,num_bedrooms,agent_logo
0,https://www.zoopla.co.uk/for-sale/details/6388...,01455 886081,Attractive extended traditional bay fronted s...,"98 Castle Street, Hinckley",52.538830,-1.396291,LE10,gb,1,sale,Semi-detached house,63884099,2,Hinckley,"Langdale Road, Hinckley LE10",None,https://lid.zoocdn.com/354/255/c9c518b9e9bd024...,Langdale Road,Scrivins & Co Estate Agents & Letting Agents,Leicestershire,None,2023-02-04 11:38:12,England,2023-02-04 11:39:32,260000.0,Residential,3,https://st.zoocdn.com/zoopla_static_agent_logo...
1,https://www.zoopla.co.uk/for-sale/details/6387...,01455 364814,** viewing essential ** A beautifully present...,"112 Castle Street, Hinckley",52.548298,-1.353169,LE10,gb,4,sale,Detached house,63878342,2,Hinckley,"Bradgate Gardens, Hinckley LE10",None,https://lid.zoocdn.com/354/255/e2c80f945cd69da...,Bradgate Gardens,Castle Estates,Leicestershire,offers_over,2023-02-03 15:54:45,England,2023-02-04 09:42:14,450000.0,Residential,5,https://st.zoocdn.com/zoopla_static_agent_logo...
2,https://www.zoopla.co.uk/for-sale/details/6387...,01455 364814,**viewing essential ** A well appointed semi ...,"112 Castle Street, Hinckley",52.552856,-1.375555,LE10,gb,1,sale,Semi-detached house,63874929,2,Hinckley,"York Road, Hinckley LE10",None,https://lid.zoocdn.com/354/255/4fd00c679828a04...,York Road,Castle Estates,Leicestershire,offers_over,2023-02-03 11:20:13,England,2023-02-03 13:17:45,280000.0,Residential,3,https://st.zoocdn.com/zoopla_static_agent_logo...
3,https://www.zoopla.co.uk/for-sale/details/6387...,01455 364026,An immaculately maintained tastefully decorat...,"28-30 New Buildings, Hinckley",52.534348,-1.392864,LE10,gb,1,sale,Semi-detached house,63871971,1,Hinckley,"Strathmore Road, Hinckley LE10",None,https://lid.zoocdn.com/354/255/31105260d73c332...,Strathmore Road,Profiles,Leicestershire,offers_in_region_of,2023-02-02 22:07:06,England,2023-02-02 22:07:06,260000.0,Residential,3,https://st.zoocdn.com/zoopla_static_agent_logo...
4,https://www.zoopla.co.uk/for-sale/details/6152...,01455 364871,You're sure to be impressed when you enter thi...,"84 Castle Steet, Hinckley",52.546017,-1.386440,LE10,gb,1,sale,Bungalow,61524804,1,Hinckley,"Aulton Way, Hinckley, Leicestershire LE10",None,https://lid.zoocdn.com/354/255/88ccf710c285162...,Aulton Way,Your Move - Hinckley,Leicestershire,None,2023-02-01 18:08:21,England,2023-02-01 18:08:21,315000.0,Residential,3,https://st.zoocdn.com/zoopla_static_agent_logo...


In [7]:
zoopla_df.shape

(292, 28)

### Save to csv file

In [8]:
try:
    os.mkdir(SAVE_FOLDER)
except OSError:
    pass

save_file = os.path.join(SAVE_FOLDER, f'zoopla_properties_{AREA.lower()}.csv')
    
zoopla_df.to_csv(save_file, index=False)